# DataWorkshop Warsaw SMOG Project
---

### 1. Zmiana problemu regresji liniowej na klasyfikację
### 2. Przykład użycia Pipeline do automatyzacji testów modeli

In [3]:
# Importowanie blibliotek
import numpy as np
import pandas as pd

# Pipeline
from sklearn.pipeline import Pipeline

# Scaler
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

# GridSearchCV etc.
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

# Score
from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_csv('pm25.csv',index_col = 0, header = 0)
df = df.drop('pm25lag', axis = 1) # usunięcie zmiennej pm25 opóźnionej

feats = ['Rok', 'Miesiąc', 'Dzień', 'Godzina',
         'Zachmurzenie_ogólne_[oktanty]',
         'Kierunek_wiatru_[°]',
         'Prędkość_wiatru_[m/s]',
         'Temperatura_powietrza_[°C]',
         'Wilgotność_względna_[%]',
         'Ciśnienie_na_pozimie_stacji_[hPa]',
         'Ciśnienie_na_pozimie_morza_[hPa]',
         'dzien_tygodnia']

In [5]:
df['Niepodległości'].hist(bins=100)

In [6]:
# Zmiana problemu regresji liniowej na klasyfikację

# Transformacja kolumny 'Niepodległości' na wartości binarne
# Według norm WHO z 2017 roku dla średniej dobowej:
# "nie ma smogu" : 0 (przedział < 25 µg)
# "jest smog" : 1 (przedział >= 25 µg) 

df.loc[df['Niepodległości'] < 25, 'Niepodległości'] = 0
df.loc[df['Niepodległości'] >= 25, 'Niepodległości'] = 1

df['Niepodległości'].value_counts()

0.0    25785
1.0    18039
Name: Niepodległości, dtype: int64

In [5]:
y = df['Niepodległości']
X = df[feats].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

In [9]:
# Przykład użycia Pipeline do automatyzacji testów modeli

# Definiowanie listy pipelineów: każdy pipe to jeden model
# W każdym modelu (pipe) definiowanie parametrów do GridSearchCV

modele = [
    (Pipeline([("scaler", StandardScaler()), ("regresja", LogisticRegression(solver="liblinear"))]),
    {
    "regresja__penalty": ["l1", "l2"],
    "regresja__C": 10.0**np.arange(-2, 2, 1)}),
    
    (Pipeline([("drzewo", DecisionTreeClassifier())]),
    {"drzewo__criterion": ["gini", "entropy"],
    "drzewo__max_depth": [2, 5, 10, 15, None]}),
   
    (Pipeline([("bagging", BaggingClassifier())]),
     {"bagging__base_estimator": [LogisticRegression(C=0.1, solver="liblinear"), DecisionTreeClassifier()],
     "bagging__n_estimators": [1,10],
     "bagging__max_features": [0.2, 0.8]}
    ),
    
    (Pipeline([("randomForest", RandomForestClassifier())]),
     {"randomForest__n_estimators": [1,10,100]}
    )
    
    ]

In [10]:
# Dla każdej pary pipeline-parametry:
# 1. GridSearchCV
# 2. Zapamiętaj najlepszy model
najlepsze = []
for model, parametry in modele:
    print(model.steps[-1][0])
    gs = GridSearchCV(model, parametry, cv=5, verbose=1, n_jobs=-1)
    gs.fit(X_train, y_train)
    najlepsze.append((gs.best_estimator_, gs.best_params_))

regresja
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


drzewo
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.3s finished


bagging
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  40 | elapsed:    1.8s remaining:    1.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    5.5s finished


randomForest
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    7.4s finished


In [11]:
# Dla każdego najlepszego modelu:
# 1. Uruchom na teście
# 2. Wyświetlenie wyników
for model, parametry in najlepsze:
    print(model.steps[-1][0])
    print(parametry)
    print(accuracy_score(model.predict(X_test),y_test))

regresja
{'regresja__C': 1.0, 'regresja__penalty': 'l1'}
0.7550958320657134
drzewo
{'drzewo__criterion': 'entropy', 'drzewo__max_depth': None}
0.8522208700943109
bagging
{'bagging__base_estimator': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best'), 'bagging__max_features': 0.8, 'bagging__n_estimators': 10}
0.8860663218740493
randomForest
{'randomForest__n_estimators': 100}
0.9045482202616367
